In [ ]:
# [쉬움주의] 케글 타이타닉 로지스틱 회귀

from sklearn import metrics
import numpy as np

# 1단계 csv ---> 데이터 프레임으로 변환

import pandas as pd
import seaborn as sns

df = pd.read_csv("C:\\Users\\icecr\\OneDrive\\CSV_file\\titanic\\train.csv")
#df = pd.read_csv("C:\\Users\\82103\\OneDrive\\CSV_file\\titanic\\train.csv")  #for notebook
df

In [ ]:
# 컬럼이 모두다 출력될 수 있도록 출력할 열의 개수 한도를 늘리기
pd.set_option('display.max_columns',15)

In [ ]:
# 2단계 결측치 확인하고 제거하거나 치환한다.
# 2.1 타이타닉 데이터 프레임의 자료형을 확인한다.

mask4 = (df.Age<10) | (df.Sex=='female') 
df['child_women']=mask4.astype(int)
df

In [ ]:
print ( df.columns)

In [ ]:
# embark 와 embark_town 이 같은 데이터여서 embark 컬럼을 삭제해야함

rdf = df.drop(['PassengerId','Cabin','Name','Ticket'], axis =1)
rdf

#### 결측치가 있는 컬럼을 확인합니다.

In [ ]:
df.isnull().sum()

In [ ]:
# 2.4 age(나이) 열에 나이가 없는 모든행을 삭제한다.
# 데이터가 한개라도 없으면 drop 해라 (how = 'any')
# 모든 데이터가 없으면 drop 해라 (how = 'all')

print(rdf.shape)

#rdf = rdf.dropna( subset=['Age'], how='all', axis=0)
#rdf['Age']=rdf['Age'].fillna(method='ffill', inplace=True)

most_freq = rdf['Age'].value_counts(dropna=True).idxmax()  
most_freq

In [ ]:
rdf['Age'].fillna(31, inplace=True)
rdf.isnull().sum()

In [ ]:
# 2.5 embark 열의 NaN 값을 승선도시중 가장 많이 출현한 값으로 치환하기
most_freq = rdf['Embarked'].value_counts().idxmax()
print(most_freq)
rdf['Embarked'].fillna(most_freq, inplace = True)
rdf.isnull().sum()

In [ ]:
# 2.6 fare 의 이상치를 제거합니다.

local_std = rdf.Fare.std() * 5  
print(local_std)

In [ ]:
rdf = rdf[:][ rdf['Fare'] < local_std  ]

In [ ]:
len(rdf)

In [ ]:
# 3단계 범주형 데이터를 숫자형으로 변환하기
# 3.1 feature selection (분석에 필요한 속성을 선택)
#ndf = rdf[['Survived','Pclass','Sex','Age','Sibsp','Parch','Embarked','child_women']]

ndf = rdf
ndf

In [ ]:
# 선택된 컬럼중 2개(sex, embarked) 가 범주형이다.
#3.2 범주형 데이터를 숫자로 변환하기(원핫 인코딩)

gender = pd.get_dummies(ndf['Sex'])
ndf = pd.concat([ndf,gender], axis= 1)
onehot_embarked = pd.get_dummies(ndf['Embarked'])
ndf = pd.concat([ndf,onehot_embarked],axis=1)
ndf.drop(['Sex','Embarked'], axis=1, inplace = True)

In [ ]:
ndf

In [ ]:
# 4단계 정규화
# 4.1 독립변수와 종속변수(라벨) 을 지정한다.
# survived  pclass   age  sibsp  parch  female  male  C  Q  S
#   라벨                       데이터
# 종속변수                     독립변수

print(ndf.columns)
x = ndf[ ['Fare', 'Pclass', 'Age' ,'SibSp', 'Parch' ,'female' ,'male', 'C' ,'Q' ,'S','child_women'] ]
#x = ndf[ ['Fare', 'Pclass', 'Age' ,'SibSp', 'Parch' ,'female' ,'male', 'child_women', 'Parch'] ]

y = ndf['Survived'] # 종속변수
x.info()

In [ ]:
# 4.2 독립변수들을 정규화 한다.
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(x).transform(x)

In [ ]:
X

In [ ]:
# 7단계 테스트 데이터로 예측을 한다.

import pandas as pd
x_ktest = pd.read_csv("C:\\Users\\icecr\\OneDrive\\CSV_file\\titanic\\test.csv")
#ktest = pd.read_csv("C:\\Users\\82103\\OneDrive\\CSV_file\\titanic\\test.csv")
x_ktest.PassengerId

#### 테스트 데이터도 훈련 데이터와 똑같이 파생변수 추가합니다. 

In [ ]:
mask4 = (x_ktest.Age<10) | (x_ktest.Sex=='female') 
x_ktest['child_women']=mask4.astype(int)
x_ktest

In [ ]:
# 테스트 데이터도 필요없는 컬럼을 삭제합니다.

rdf_x_ktest = x_ktest.drop(['PassengerId','Cabin','Name','Ticket'], axis =1)
print(rdf_x_ktest)

In [ ]:
# 테스트 데이터의 나이의 결측치를 21로 채워넣습니다.
rdf_x_ktest['Age'].fillna(31, inplace=True)
rdf_x_ktest.isnull().sum()

In [ ]:
# 2.5 embark 열의 NaN 값을 승선도시중 가장 많이 출현한 값으로 치환하기
most_freq = rdf_x_ktest['Embarked'].value_counts().idxmax()
rdf_x_ktest['Embarked'].fillna(most_freq, inplace = True)
print(rdf_x_ktest)

In [ ]:
# 2.6 fare 열의 NaN 값을 요금중 가장 많이 출현한 값으로 치환하기
most_freq = rdf_x_ktest['Fare'].value_counts().idxmax()
rdf_x_ktest['Fare'].fillna(most_freq, inplace = True)
print(rdf_x_ktest)

In [ ]:
# 3단계 범주형 데이터를 숫자형으로 변환하기
# 3.1 feature selection (분석에 필요한 속성을 선택)
#ndf = rdf[['Survived','Pclass','Sex','Age','Sibsp','Parch','Embarked','child_women']]

ndf_x_ktest = rdf_x_ktest

In [ ]:
ndf_x_ktest

In [ ]:
# 선택된 컬럼중 2개(sex, embarked) 가 범주형이다.
#3.2 범주형 데이터를 숫자로 변환하기(원핫 인코딩)
gender = pd.get_dummies(ndf_x_ktest['Sex'])
ndf_x_ktest = pd.concat([ndf_x_ktest,gender], axis= 1)
onehot_embarked = pd.get_dummies(ndf_x_ktest['Embarked'])
ndf_x_ktest = pd.concat([ndf_x_ktest,onehot_embarked],axis=1)
ndf_x_ktest.drop(['Sex','Embarked'], axis=1, inplace = True)

In [ ]:
ndf_x_ktest

In [ ]:
# 4단계 정규화
# 4.1 독립변수와 종속변수(라벨) 을 지정한다.
# survived  pclass   age  sibsp  parch  female  male  C  Q  S
#   라벨                       데이터
# 종속변수                     독립변수
print(ndf_x_ktest.columns)
x = ndf_x_ktest[ ['Fare','Pclass', 'Age' ,'SibSp', 'Parch' ,'female' ,'male', 'C' ,'Q' ,'S', 'child_women'] ]
#x = ndf_x_ktest[ ['Fare','Pclass', 'Age' ,'SibSp', 'Parch' ,'female' ,'male', 'child_women','Parch'] ]

print ( x.isnull().sum( axis=0)  )  

In [ ]:
#y = ndf_x_ktest['Survived'] # 종속변수

# 4.2 독립변수들을 정규화 한다.
from sklearn import preprocessing
X_test = preprocessing.StandardScaler().fit(x).transform(x)
print(len(X))
print(X.shape)
print(X_test.shape)

In [ ]:
from sklearn.neural_network  import  MLPClassifier
from sklearn.model_selection import GridSearchCV

param_grid = [
        {
            'activation' : ['identity', 'logistic', 'tanh', 'relu'],
            'solver' : ['lbfgs', 'sgd', 'adam'],
            'hidden_layer_sizes': [
             (1,2),(2,2),(3,2),(4,2),(5,2),(6,2),(7,2),(8,2),(9,2),(10,2),(11,2), (12,2),(13,2),(14,2),(15,2),(16,2),(17,2),(18,2),(19,2),(20,2),(21,2)
             ]
        }
       ]
 
grid_dtree = GridSearchCV(MLPClassifier(), param_grid, cv=3, n_jobs = -1, verbose = 2 )

grid_dtree.fit( X, y)

In [ ]:
print("Best parameters set found on development set:")
print(grid_dtree.best_params_) 

In [ ]:
# 7단계 테스트 데이터로 예측을 한다.

y_hat = grid_dtree.predict( X_test )
print(y_hat)

for  i,a  in  enumerate(y_hat):
    print (str(i+892) + ',' + str(a))
    
    

In [ ]:
x_ktest

In [ ]:
test_id =x_ktest.PassengerId
pred = y_hat
test_id

In [ ]:
submission = pd.DataFrame({'PassengerId':test_id,'Survived':pred})
submission.to_csv("C:\\Users\\icecr\\OneDrive\\CSV_file\\titanic\\Sub_01.csv",index=False)
#submission.to_csv("C:\\Users\\82103\\OneDrive\\CSV_file\\titanic\\Sub_01.csv",index=False)
submission.head()
